In [28]:
# Set up folders
from EDA_functions import folders_set_up
import os

# Work with datarames
import pandas as pd
import numpy as np

# Evaluate models
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np

from IPython.display import Image

from surprise import Dataset
from surprise import Reader
from surprise import SVD # Matrix Factorization, Single Value Decomposition
from surprise.model_selection import cross_validate

#from scipy.sparse import spmatrixc

### Folders
Run the code below if you have the following structure:
- Group-project: GitHub folder
- 01 Input
- 02 Output

In [2]:
analysis_folder, input_folder, output_folder = folders_set_up.generate_folders()

In [13]:
# Import ratings data
title_level_df = pd.read_csv(os.path.join(input_folder,'Books_rating.csv'))

title_level_df.columns

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

In [9]:
# Import train indices
train_indices = pd.read_csv(
    os.path.join(output_folder, "train_indices.csv")
)

train_indices.columns

Index(['Unnamed: 0', 'Title', 'index'], dtype='object')

In [14]:
# Keep ratings of books in the training dataset
recommender_df = pd.merge(
    train_indices,
    title_level_df,
    on = 'Title',
    how = 'left'
)

recommender_df.shape

(800337, 12)

In [22]:
recommender_df['Title'].nunique()

21419

In [15]:
recommender_df.columns

Index(['Unnamed: 0', 'Title', 'index', 'Id', 'Price', 'User_id', 'profileName',
       'review/helpfulness', 'review/score', 'review/time', 'review/summary',
       'review/text'],
      dtype='object')

In [29]:
# Drop variables

# https://surprise.readthedocs.io/en/stable/reader.html
reader = Reader(rating_scale=(0.5, 4.0))
#reader = Reader()

data = Dataset.load_from_df(recommender_df[['Id', 'Title', 'review/score']], reader)


In [31]:
svd = SVD()

cross_validate(
    svd, 
    data, 
    measures=['RMSE', 'MAE', 'MSE'], # RMSE - Root Mean Square Error (MSE)
    cv=3, 
    verbose=True
)

Evaluating RMSE, MAE, MSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.1751  1.1772  1.1766  1.1763  0.0009  
MAE (testset)     0.9748  0.9762  0.9748  0.9753  0.0007  
MSE (testset)     1.3810  1.3858  1.3843  1.3837  0.0020  
Fit time          2.40    2.51    2.72    2.55    0.13    
Test time         0.89    0.99    0.86    0.91    0.05    


{'test_rmse': array([1.1751427 , 1.17722043, 1.1765651 ]),
 'test_mae': array([0.97476733, 0.97623271, 0.97476084]),
 'test_mse': array([1.38096036, 1.38584795, 1.38430543]),
 'fit_time': (2.4044129848480225, 2.5144309997558594, 2.7174739837646484),
 'test_time': (0.8920009136199951, 0.9874722957611084, 0.8648440837860107)}